# 多输入多输出通道
实现一个多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 利用zip对输入通道做遍历
    # 对各个通道依次卷积再相加
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


验证互相关运算的输出

In [7]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], 
                 [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
print(X.shape)
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(K.shape)

corr2d_multi_in(X, K)

torch.Size([2, 3, 3])
torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

In [8]:
def corr2d_multi_in_out(X, K):
    # 在0这个维度上堆起来
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape
# 输入是3，输出是2，高和宽分别是2。共有六个核

torch.Size([3, 2, 2, 2])

In [9]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1 * 1卷积（等价为全连接层）

In [12]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

关于torch.stack()

In [11]:
T1 = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

T2 = torch.tensor([[10, 20, 30], [40, 50, 60], [70, 80, 90]])

T3 = torch.stack((T1, T2), 0)

T4 = torch.stack((T1, T2), 1)

T5 = torch.stack((T1, T2), 2)

torch.Size([3, 3])
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
torch.Size([3, 3])
tensor([[10, 20, 30],
        [40, 50, 60],
        [70, 80, 90]])
torch.Size([2, 3, 3])
tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 20, 30],
         [40, 50, 60],
         [70, 80, 90]]])
torch.Size([3, 2, 3])
tensor([[[ 1,  2,  3],
         [10, 20, 30]],

        [[ 4,  5,  6],
         [40, 50, 60]],

        [[ 7,  8,  9],
         [70, 80, 90]]])
torch.Size([3, 3, 2])
tensor([[[ 1, 10],
         [ 2, 20],
         [ 3, 30]],

        [[ 4, 40],
         [ 5, 50],
         [ 6, 60]],

        [[ 7, 70],
         [ 8, 80],
         [ 9, 90]]])
